In [ ]:
# Subject-wise Performance Evaluation
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, f1_score

# Configuration
RESULT_DIR = Path("../Results/Experiment_Result/Model_Preds/Original")
SUBJECT_IDS = [str(i) for i in range(201, 211)]  # '201' to '210'
OUTPUT_CSV = RESULT_DIR / "subject_performance.csv"

def evaluate_subject(sid: str) -> dict:
    """
    Load the first CSV in subject folder, compute BA and Macro F1.
    Returns dict with 'BA' and 'MacroF1' percentages.
    """
    subj_dir = RESULT_DIR / sid
    if not subj_dir.is_dir():
        return {}
    csv_files = sorted([f for f in subj_dir.glob('*.csv')])
    if not csv_files:
        return {}
    df = pd.read_csv(csv_files[0])
    if not {'y_true', 'y_pred'}.issubset(df.columns):
        return {}
    ba = balanced_accuracy_score(df['y_true'], df['y_pred']) * 100
    f1 = f1_score(df['y_true'], df['y_pred'], average='micro') * 100
    return {'BA': round(ba, 2), 'MacroF1': round(f1, 2)}

if __name__ == '__main__':
    # Evaluate each subject
    metrics = {}
    all_true, all_pred = [], []
    for sid in SUBJECT_IDS:
        res = evaluate_subject(sid)
        if res:
            metrics[sid] = res
            df = pd.read_csv((RESULT_DIR / sid / sorted((RESULT_DIR / sid).glob('*.csv'))[0]))
            all_true.extend(df['y_true'])
            all_pred.extend(df['y_pred'])

    # Create DataFrame
    df_metrics = pd.DataFrame(metrics).T

    # Add overall mean
    overall = pd.Series({
        'BA': round(pd.Series(all_true).pipe(balanced_accuracy_score, pd.Series(all_pred)) * 100, 2),
        'MacroF1': round(f1_score(all_true, all_pred, average='micro') * 100, 2)
    }, name='Mean')
    df_metrics = df_metrics.append(overall)

    # Save to CSV
    df_metrics.to_csv(OUTPUT_CSV, index=True)
    print(f"Saved per-subject performance to {OUTPUT_CSV}")
